In [1]:
import os
import numpy as np
from PIL import Image 
import random
import scipy.io
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score



In [ ]:
# Labels into numpy array
path = 'E:\whistle_identification\DOLPHI\labels_2_48kHz.mat'
mat_content = scipy.io.loadmat(path)

labels = mat_content['labels']
newLabels = []
for index,i in enumerate(labels):
    newLabels.append(i[0])
    
np.save('E:\whistle_identification\DOLPHI\\labels48kHz.npy',newLabels)



In [ ]:
# Data path

imagesDir = 'E:\whistle_identification\DOLPHI\images48kHz'
images = os.listdir(imagesDir)
labels1 = np.load('E:\whistle_identification\DOLPHI\\labels48kHz.npy')
labels1 = labels1.astype(np.int32) 
normImages = []


In [ ]:
# Reorder label vector according to the order in which image files are read via 'os.listdir()'
labels = [0]*len(images)

for i,image in enumerate(images):
    index = image.split('r')[1]
    index = index.split('.')[0]
    index = int(index)-1
    labels[i] = labels1[index]

labels = np.array(labels)


In [ ]:
# Scrolls through image vectors for normalizing and resizing images

for image in images:
    
    img = Image.open(os.path.join(imagesDir,image))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis = 2)
    img_array = np.repeat(img_array,3, axis = 2)
    img_normalized = img_array/255.0
    img_normalized = img_normalized.astype(np.float32)
    normImages.append(img_normalized)

In [ ]:
# Class balancing

print('Class balancing...')
X = []
y = []
positivi = 0 
negativi = 0 

for ind,y1 in enumerate(labels):
    if y1 != 0 :
        positivi = positivi + 1
        X.append(normImages[ind])
        y.append(y1)


for index,y2 in enumerate(labels):

    if negativi<positivi and y2 == 0:
        X.append(normImages[index])
        y.append(y2)
        negativi = negativi + 1
    elif negativi == positivi:
        break

In [ ]:
# Data shuffling

combined = list(zip(X,y))
random.shuffle(combined)

X, y = zip(*combined)

In [ ]:
# Division into training and testing set

lengthX = len(X)
lengthY = len(y)


rest_X = X[int(lengthX * 0.9):]
X_train= X[:int(lengthX * 0.9)]
X_train_tot= np.array(X_train)
rest_y = y[int(lengthY * 0.9):]
y_train = y[:int(lengthY * 0.9)]     
y_train_tot = np.array(y_train)  

In [ ]:
# 5-Fold Cross Validation

#####################################################################################
# Model definition
print('Definizione modello...')
model = Sequential()

# Adding Convolutional and Max Pooling Layers
model.add(Conv2D(32, (7, 7), strides=(2, 2), activation='relu', input_shape=(224,223,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (5, 5), strides=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Data flattening
model.add(Flatten()) 

# Adding Fully Connected layers
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Model build
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
###################################################################################

""" X = np.load('E:\whistle_identification\DOLPHI\\total_trainX.npy',allow_pickle=True)
y = np.load('E:\whistle_identification\DOLPHI\\total_trainY.npy',allow_pickle=True) """
X = X_train_tot
y = y_train_tot

k = 5  # Number of folds
stratified_kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=1)

best_accuracy = 0
best_fold = None
fold = 1
for train_index, val_index in stratified_kfold.split(X, y):
    print(f"Fold: {fold}")
    
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    # Train model
    model.fit(X_train, y_train)
    
    # Validate model
    y_pred = model.predict(X_val)
    y_pred1 = []
    for i in y_pred:
        if i[0]>= 0.5:
            y_pred1.append(1)
        else:
            y_pred1.append(0)

    y_pred1 = np.array(y_pred1)
 
    accuracy = accuracy_score(y_val, y_pred1)
    
    print(f"Validation Accuracy: {accuracy}\n")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_fold = (X_train, X_val, y_train, y_val)
    
    fold += 1

print(f"Best accuracy: {best_accuracy}")
print(f"Best fold: {best_fold}")

# Save training and validation data for Vanilla CNN as '.npy' files
""" np.save('E:\whistle_identification\DOLPHI\\trainImages_vanilla_48kHz.npy', best_fold[0])
np.save('E:\whistle_identification\DOLPHI\\trainLabels_vanilla_48kHz.npy', best_fold[2])
np.save('E:\whistle_identification\DOLPHI\\valImages_vanilla_48kHz.npy', best_fold[1])
np.save('E:\whistle_identification\DOLPHI\\valLabels_vanilla_48kHz.npy', best_fold[3]) """

In [ ]:
# 5-Fold Cross Validation

#####################################################################################
# Model definition

img_width,img_height = 224,223
epochs = 1
batch_size = 16

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the weights of the VGG16
for layer in base_model.layers:
    layer.trainable = False

# Add fully connected layers at the end of the model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

for layer in model.layers[-3:]:
    layer.trainable = True

# Model build
# Model build
opt = tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
###################################################################################

""" X = np.load('E:\whistle_identification\DOLPHI\\total_trainX.npy',allow_pickle=True)
y = np.load('E:\whistle_identification\DOLPHI\\total_trainY.npy',allow_pickle=True) """
X = X_train_tot
y = y_train_tot

k = 5  # Number of folds
stratified_kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=1)

best_accuracy = 0
best_fold = None
fold = 1
for train_index, val_index in stratified_kfold.split(X, y):
    print(f"Fold: {fold}")
    
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    # Train model
    model.fit(X_train, y_train)
    
    # Validate model
    y_pred = model.predict(X_val)
    y_pred1 = []
    for i in y_pred:
        if i[0]>= 0.5:
            y_pred1.append(1)
        else:
            y_pred1.append(0)

    y_pred1 = np.array(y_pred1)
 
    accuracy = accuracy_score(y_val, y_pred1)
    
    print(f"Validation Accuracy: {accuracy}\n")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_fold = (X_train, X_val, y_train, y_val)
    
    fold += 1

print(f"Best accuracy: {best_accuracy}")
print(f"Best fold: {best_fold}")

# Save training and validation data for VGG16 as '.npy' files
""" np.save('E:\whistle_identification\DOLPHI\\trainImages_vgg_48kHz.npy', best_fold[0])
np.save('E:\whistle_identification\DOLPHI\\trainLabels_vgg_48kHz.npy', best_fold[2])
np.save('E:\whistle_identification\DOLPHI\\valImages_vgg_48kHz.npy', best_fold[1])
np.save('E:\whistle_identification\DOLPHI\\valLabels_vgg_48kHz.npy', best_fold[3]) """

In [ ]:
# Save test data as '.npy' files

""" np.save('E:\whistle_identification\DOLPHI\\testX48kHz.npy', rest_X)
np.save('E:\whistle_identification\DOLPHI\\testY48kHz.npy', rest_y) """